# Challenge 09/12/2024

Challenge instructions [here](https://adventofcode.com/2024/day/9)

## Highlights & Notes

- Goal:


## Setup & Imports


In [1]:
import numpy as np
import sys
import os

# Get utility functions
sys.path.append(os.path.abspath('../utils'))
from utils import read_input_file, remove_newline_char, split_lines

# Quick ANSI color code shortcuts
r = "\033[31m";y = "\033[33m";g = "\033[32m";b = "\033[34m";e = "\033[0m"

## Part 1


In [ ]:
def generate_initial_blocks(fileSizes, freeSpaces):
    # Generate the initial block list
    blocks = []
    fileID = 0
    nbFiles = len(fileSizes)
    nbFreeSpaces = len(freeSpaces)

    # Since the disk map alternates between files and free spaces,
    # we can iterate over the max of nbFiles and nbFreeSpaces
    idxFile = 0
    idxFreeSpace = 0

    while idxFile < nbFiles or idxFreeSpace < nbFreeSpaces:
        # Add file blocks
        if idxFile < nbFiles:
            size = fileSizes[idxFile]
            blocks.extend([fileID] * size)
            fileID += 1
            idxFile += 1
        # Add free space blocks
        if idxFreeSpace < nbFreeSpaces:
            size = freeSpaces[idxFreeSpace]
            blocks.extend(['.'] * size)
            idxFreeSpace += 1
    return blocks

def compact(blocks):
    compactedBlocks = blocks.copy()
    # Move file blocks from the end to the leftmost free spaces
    left = 0
    right = len(blocks) - 1
    while left < right:
        # Find the next free space from the left
        while left < right and blocks[left] != '.':
            left += 1
        # Find the next file block from the right
        while left < right and blocks[right] == '.':
            right -= 1
        if left < right:
            # Move the file block to the free space
            blocks[left] = blocks[right]
            blocks[right] = '.'
            left += 1
            right -= 1
    return compactedBlocks

def compute_checksum(blocks):
    checksum = 0
    for position, block in enumerate(blocks):
        if block != '.':
            checksum += position * block  # block is the file ID
    return checksum

originalDiskSpace = read_input_file("input.txt")[0] # File with only 1 line
# print(originalDiskSpace)
fileSizes = [int(digit) for digit in originalDiskSpace[::2]]
freeSpaces = [int(digit) for digit in originalDiskSpace[1::2]]
# print(fileSizes)
# print(freeSpaces)
blocks = generate_initial_blocks(fileSizes, freeSpaces)
compactedBlocks = compact(blocks)
checksum = compute_checksum(compactedBlocks)
print(f"Filesystem checksum: {checksum}")

SyntaxError: expected ':' (2977159790.py, line 47)

## Part 2

Instructions [here](https://adventofcode.com/2024/day/2#part2)


In [ ]:
def read_input_file(filename):
    with open(filename, 'r') as f:
        return f.read().strip()

def parse_disk_map(disk_map_line):
    # Extract file sizes and free space sizes using slicing
    file_sizes = [int(digit) for digit in disk_map_line[::2]]
    free_spaces = [int(digit) for digit in disk_map_line[1::2]]
    return file_sizes, free_spaces

def generate_initial_blocks(file_sizes, free_spaces):
    # Generate the initial block list
    blocks = []
    file_id = 0
    nb_files = len(file_sizes)
    nb_free_spaces = len(free_spaces)
    idx_file = 0
    idx_free_space = 0

    while idx_file < nb_files or idx_free_space < nb_free_spaces:
        # Add file blocks
        if idx_file < nb_files:
            size = file_sizes[idx_file]
            blocks.extend([file_id] * size)
            file_id += 1
            idx_file += 1
        # Add free space blocks
        if idx_free_space < nb_free_spaces:
            size = free_spaces[idx_free_space]
            blocks.extend(['.'] * size)
            idx_free_space += 1
    return blocks

def get_files(blocks):
    # Extract file information: file ID, start, end, size
    files = []
    i = 0
    n = len(blocks)

    while i < n:
        if blocks[i] != '.':
            # Start of a file
            file_id = blocks[i]
            start = i
            while i < n and blocks[i] == file_id:
                i += 1
            end = i - 1  # Inclusive
            size = end - start + 1
            files.append({'file_id': file_id, 'start': start, 'end': end, 'size': size})
        else:
            i += 1
    return files

def simulate_compaction(blocks, files):
    # Sort files by decreasing file ID
    files.sort(key=lambda x: -x['file_id'])
    for file in files:
        file_size = file['size']
        # Build the list of free spaces to the left of the file
        free_spaces = []
        i = 0
        n = file['start']
        while i < n:
            if blocks[i] == '.':
                # Start of free space
                start = i
                while i < n and blocks[i] == '.':
                    i += 1
                end = i - 1
                size = end - start + 1
                free_spaces.append({'start': start, 'end': end, 'size': size})
            else:
                # Skip over files
                while i < n and blocks[i] != '.':
                    i += 1
        # Find free spaces that can fit the file
        eligible_free_spaces = [fs for fs in free_spaces if fs['size'] >= file_size]
        if eligible_free_spaces:
            # Choose the leftmost free space
            target_fs = min(eligible_free_spaces, key=lambda x: x['start'])
            # Move the file to that free space
            # Remove the file from its old position
            for idx in range(file['start'], file['end'] + 1):
                blocks[idx] = '.'
            # Place the file in the new position
            new_start = target_fs['start']
            new_end = new_start + file_size - 1
            for idx in range(new_start, new_end + 1):
                blocks[idx] = file['file_id']
            # Update the file's start and end positions
            file['start'] = new_start
            file['end'] = new_end
            # No need to update free_spaces since we rebuild them each time
        else:
            # Could not move the file
            pass
    return blocks

def compute_checksum(blocks):
    # Compute the filesystem checksum
    checksum = 0
    for position, block in enumerate(blocks):
        if block != '.':
            checksum += position * block  # block is the file ID
    return checksum

# Main execution
disk_map_line = read_input_file("input.txt")

# Parse the disk map
file_sizes, free_spaces = parse_disk_map(disk_map_line)

# Generate the initial block list
blocks = generate_initial_blocks(file_sizes, free_spaces)

# Get the list of files with their positions
files = get_files(blocks)

# Simulate the compaction process
blocks = simulate_compaction(blocks, files)

# Compute the checksum
checksum = compute_checksum(blocks)
print(f"Filesystem checksum: {checksum}")